In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from pandas import DataFrame as df
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import get_decimal_separator
from Data_setup_ import data_set
import os
from galvani import BioLogic
from math import floor,log10

fig_title = 'Title'
currentcorrect = 1000 #converts to mA to A in the log
areacorrect = 1 #converts the surface area in the log
#start figure, assign colors 
fig = go.Figure()
fig.update_xaxes(range=[-5.5, -1.5],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[0, 1.2],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
colors = px.colors.qualitative.Dark24

fig.update_layout(
    xaxis_title="Log Current (A/cm<sup>2</sup>)",
    yaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    margin={'t':0,'l':0,'b':0,'r':0},
    legend_font_size=22,
    font=dict(
        family="Times New Roman, monospace",
        size=22,
        color="dimgrey",
    ),
)
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(255, 255, 255, 255)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})

# #find the lowest value between all the fit_max, for using it for getting the overpotential at that current
# min_fit_max = float('inf')
# for i in data_set:
#     if('fit_max' in data_set[i]):
#         min_fit_max = min(min_fit_max, data_set[i]['fit_max'])
# # round it down to one significant digit
# min_fit_max = log10( floor( pow(10,min_fit_max)/pow(10,floor(min_fit_max)) ) * pow(10,floor(min_fit_max)) )

#load in data either .mpr or .mpt
for i in data_set:
    
    file = data_set[i]['data_Tafel']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    if not os.path.exists(data_set[i]['data_Tafel']):
        continue
    
    #define variables, sets up number of experiments, does the voltage correction based on Ru and 
    exp = data['Ns']
    Nexp = exp[-1:] + 1
    voltage = data['Ewe/V']
    current = data['control/mA']
    #current = current/data_set[i]['mass']
    voltage_corrected = data['Ewe/V'] - data_set[i]['cal'] - data['control/mA'] * data_set[i]['Ru']/1000  #does the iR correction converts to overpotential
    #creates numpy array Tafel with the voltage, voltage error, and logJ from the data 
    j = 0
    Tafel = []
    for j in range(int(Nexp)):
        v = df.mean(voltage_corrected.loc[exp==j])
        verr = df.std(voltage_corrected.loc[exp==j])
        J = current / currentcorrect / areacorrect
        logJ = np.log10(J.loc[exp==j])
        logJ = logJ.iloc[0:1] 
        logJ = float(logJ)
        logJ = np.round(logJ,2)
        Tafel.append([v, verr, logJ])   
    Tafel = np.array(Tafel)
    
    #calculate fits 
    
    x = np.array(Tafel[:,2])
    y = np.array(Tafel[:,0])
    fit_min = data_set[i]['fit_min']
    fit_max = data_set[i]['fit_max']
    mask = (x >= fit_min) & (x <= fit_max)
    x_fit = x[mask]
    y_fit = y[mask]
    z, cov = np.polyfit(x_fit, y_fit, 1, cov=True)
    slope = z[0]
    intercept = z[1]
    slope_error = np.sqrt(cov[0,0])
    intercept_error = np.sqrt(cov[1,1])
    
    # get the overpotential at the current defined above
    #y_at_min_fit_max = np.polyval(z, min_fit_max)
    
    # print(f"{data_set[i]['label']}: slope={slope*1000:.0f}±{slope_error*1000:.0f} mV/decade, " +
    #     f"Y-intercept={intercept:.2f}±{intercept_error:.2f}, "+
    #     f"overpotential at 10^{min_fit_max:.2f} ({pow(10,min_fit_max):.{1}e}) A/cm2={y_at_min_fit_max:.3f}")
    
    x_pred = np.linspace(fit_min, fit_max, num=50)
    y_pred = np.polyval(z, x_pred)
    fig.add_trace(go.Scatter(
        x=x_pred,
        y=y_pred,
        mode='lines',
        showlegend= False,
        line_color=colors[data_set[i]['color_index']]
))

    fig.add_trace(go.Scatter(
        x=Tafel[:,2],
        y=Tafel[:,0],
        error_y=dict(
            type='data',
            
            array=Tafel[:,1],
            visible=True),
        mode='markers',
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
        
    )
   
)


## convert your array into a dataframe
fig.update_layout(
    width = 500,
    height = 500,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0,
    font=dict(
        family="Times New Roman, monospace",
        size=22,
        color="dimgrey",
    ),
    )
)    
fig.show()
fig.write_image(fig_title +' Tafel.jpeg')
fig.write_image(fig_title + ' Tafel.svg')
